In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from glob import glob

In [8]:
import os
from typing import List, Dict  # List와 Dict 타입 힌트를 위한 임포트

def generate_manual_index(root_dir: str, manual_type: str = "user") -> List[Dict]:
    """
    # 함수 설명
    지정된 루트 디렉토리에서 하위 메뉴얼 디렉토리를 순회하며,
    .md 파일과 img 폴더의 이미지들을 분석하여 지정된 JSON 형태로 반환합니다.

    # 매개변수 설명
    Args:
        root_dir (str): 메뉴얼 파일들이 있는 최상위 디렉토리 경로 (예: './firstUser')
        manual_type (str): 메뉴얼 종류 구분자 (예: 'user', 'admin', 'firstUser')
                          기본값은 "user"로 설정됨

    # 반환값 설명
    Returns:
        List[Dict]: 메뉴얼 정보를 담은 딕셔너리들의 리스트
    """
    # 결과를 저장할 빈 리스트 생성
    result = []
    
    # 입력받은 경로의 마지막 디렉토리 이름만 추출 
    # 예: './firstUser' -> 'firstUser'
    base_dir_name = os.path.basename(os.path.normpath(root_dir))

    # root_dir 내의 모든 항목을 순회
    for entry in os.scandir(root_dir):
        # 디렉토리인 경우에만 처리
        if entry.is_dir():
            # 현재 디렉토리 이름을 메뉴얼 타입으로 사용
            manu_type = entry.name
            
            # 메뉴얼 파일 경로와 이미지 디렉토리 경로 생성
            md_path = os.path.join(entry.path, f"{manu_type}.md")
            img_dir = os.path.join(entry.path, "img")

            # .md 파일이 없으면 다음 디렉토리로 넘어감
            if not os.path.isfile(md_path):
                continue

            # 이미지 URL들을 저장할 리스트 생성
            image_urls = []
            
            # 이미지 디렉토리가 존재하는 경우
            if os.path.isdir(img_dir):
                # 이미지 디렉토리 내 모든 파일을 정렬하여 순회
                for img_file in sorted(os.listdir(img_dir)):
                    # 이미지 파일 확장자 체크
                    if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                        # 이미지 URL 생성하여 리스트에 추가
                        image_urls.append(
                            f"https://doc.tg-cloud.co.kr/manual/console/{base_dir_name}/{manu_type}/img/{img_file}"
                        )

            # 수집된 정보를 딕셔너리 형태로 구성하여 결과 리스트에 추가
            result.append({
                "text": f"{manu_type}.md",  # 메뉴얼 파일명
                "metadata": {  # 메타데이터 정보
                    "manual_type": manual_type,  # 상위 메뉴얼 타입
                    "manu_type": manu_type,      # 현재 메뉴얼 타입
                    "source_url": f"https://doc.tg-cloud.co.kr/manual/console/{base_dir_name}/{manu_type}/{manu_type}",  # 소스 URL
                    "image_url": image_urls      # 수집된 이미지 URL 리스트
                }
            })

    # 최종 결과 반환
    return result


In [4]:
import os
import re
from typing import List, Dict

def generate_chunks_from_index(index_list: List[Dict], base_dir: str) -> List[Dict]:
    """
    Markdown 파일을 직접 줄 단위로 읽고 섹션(##) 기준으로 나누어 chunk 생성
    - 목차 섹션은 제외
    - ![](img/xxx.png) 패턴에서 이미지 파일명 추출하여 image_url 매핑
    """
    # 코드 리뷰:
    # 1. 함수의 목적과 입출력이 명확하게 정의되어 있습니다.
    # 2. 타입 힌팅을 사용하여 코드의 가독성과 유지보수성이 좋습니다.
    # 3. 문서화가 잘 되어있어 함수의 동작을 이해하기 쉽습니다.

    all_chunks = []

    for item in index_list:
        # 필요한 메타데이터를 추출하는 부분이 깔끔하게 구성되어 있습니다.
        md_filename = item["text"]
        manu_type = item["metadata"]["manu_type"]
        manual_type = item["metadata"]["manual_type"]
        source_url = item["metadata"]["source_url"]
        image_url_list = item["metadata"]["image_url"]

        md_path = os.path.join(base_dir, manu_type, md_filename)
        if not os.path.isfile(md_path):
            print(f"⚠️ 파일 없음: {md_path}")
            continue

        with open(md_path, "r", encoding="utf-8") as f:
            lines = f.readlines()

        # 초기 상태 설정이 명확합니다.
        current_section = "__intro__"
        current_text_lines = []
        skip_section = False

        # 이미지 URL 처리를 위한 내부 함수가 잘 모듈화되어 있습니다.
        def resolve_image_urls(text: str) -> List[str]:
            matches = re.findall(r'!\[.*?\]\((.*?)\)', text)
            urls = []
            for filename in matches:
                for full_url in image_url_list:
                    if filename in full_url:
                        urls.append(full_url)
            return urls

        # 개선 제안:
        # 1. 큰 파일 처리를 위해 파일을 한 번에 읽지 않고 라인별로 처리하는 것이 좋을 수 있습니다.
        # 2. 에러 처리(try-except)를 추가하면 좋을 것 같습니다.
        # 3. resolve_image_urls 함수의 성능 최적화가 필요할 수 있습니다.
        
        for line in lines:
            if line.strip().startswith("## "):
                if current_text_lines and not skip_section:
                    full_text = "".join(current_text_lines).strip()
                    if full_text:
                        chunk = {
                            "text": full_text,
                            "metadata": {
                                "manual_type": manual_type,
                                "manu_type": manu_type,
                                "section": current_section,
                                "source_url": source_url,
                                "image_url": resolve_image_urls(full_text)
                            }
                        }
                        all_chunks.append(chunk)

                current_section = line.strip().replace("##", "").strip()
                
                # skip_section = (current_section == "목차")
                # 개선된 코드
                skip_section = any(keyword in current_section.lower() for keyword in ["목차", "contents", "table of contents"])

                current_text_lines = []
            else:
                if not skip_section:
                    current_text_lines.append(line)

        # 마지막 섹션 처리 - 코드 중복이 있으므로 함수로 분리하면 좋을 것 같습니다.
        if current_text_lines and not skip_section:
            full_text = "".join(current_text_lines).strip()
            if full_text:
                chunk = {
                    "text": full_text,
                    "metadata": {
                        "manual_type": manual_type,
                        "manu_type": manu_type,
                        "section": current_section,
                        "source_url": source_url,
                        "image_url": resolve_image_urls(full_text)
                    }
                }
                all_chunks.append(chunk)

    return all_chunks


In [ ]:
# DataFrame 구조 확인
print("📊 DataFrame 기본 정보:")
print(f"• 행 수: {len(df)}")
print(f"• 열 수: {len(df.columns)}")
print(f"• 열 이름: {list(df.columns)}")

print("\n📋 DataFrame 상위 5개 행:")
print(df.head())

print("\n🔍 각 열의 데이터 타입:")
print(df.dtypes)

print("\n📝 텍스트 길이 분포:")
if 'text' in df.columns:
    df['text_length'] = df['text'].str.len()
    print(f"• 평균 텍스트 길이: {df['text_length'].mean():.1f} 문자")
    print(f"• 최소 텍스트 길이: {df['text_length'].min()} 문자")
    print(f"• 최대 텍스트 길이: {df['text_length'].max()} 문자")

print("\n🏷️ 메뉴얼 타입별 분포:")
if 'metadata.manu_type' in df.columns:
    print(df['metadata.manu_type'].value_counts())

print("\n📑 섹션별 분포:")
if 'metadata.section' in df.columns:
    print(df['metadata.section'].value_counts())


In [5]:
# 1. 메뉴얼 인덱스 생성
manual_index = generate_manual_index("./manual/user/firstUser", manual_type="firstUser")

# 2. 청크 생성
chunks = generate_chunks_from_index(manual_index, base_dir="./manual/user/firstUser")

# 3. 결과 확인
print(f"총 {len(chunks)}개 청크 생성됨")
# for chunk in chunks:
#     display(json.dumps(chunk, indent=2, ensure_ascii=False))
#     print("-" * 80)  # 구분선 추가

import pandas as pd
# 열(column) 너비 제한 해제
pd.set_option('display.max_colwidth', None)

# (선택) 출력 행 개수나 전체 열 개수도 확장 가능
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

df = pd.json_normalize(chunks)
df

총 16개 청크 생성됨


,text,metadata.manual_type,metadata.manu_type,metadata.section,metadata.source_url,metadata.image_url
0,"# Namespaces\n\n> Namespaces는 상단의 클러스터에서 서비스중인 Namespace 목록을 확인하고 생성, 삭제하는 서비스 입니다.&#x20;\n\n---",firstUser,namespaces,__intro__,https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/namespaces,[]
1,1. [Namespace 메뉴 진입](#1-namespace-메뉴-진입)\n2. [namespace 생성 신청](#2-namespace-생성-신청)\n3. [namespace 생성 확인](#3-namespace-생성-확인),firstUser,namespaces,**목차**,https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/namespaces,[]
2,1. 좌측 메뉴 `Namespaces` 메뉴 클릭\n\n ![](img/namespace_menu.png)\n\n - 좌측 메뉴 `Namespaces` 메뉴 클릭 후 네임스페이스 화면에 진입합니다.\n - 신규 프로젝트에 할당된 유저이며 프로젝트 하위에 속한 네임스페이스가 없기 때문에 빈 리스트로 표현됩니다.\n\n---,firstUser,namespaces,1. Namespace 메뉴 진입,https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/namespaces,[https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/img/namespace_menu.png]
3,"1. `생성` 버튼을 클릭하면 네임스페이스 신청을 할 수 있습니다. 네임스페이스 생성은 바로 생성되는것이 아닌 해당 `생성` 버튼을 통하여 신청 후 운영자/프로젝트 관리자의 승인을 거친 후에 네임스페이스를 생성할 수 있습니다.\n - 네임스페이스 신청 후 생성까지의 결재라인은 아래와 같이 이루어 집니다.\n\n > 유저 네임스페이스 신청 > portal 운영자 1차 결재 승인 > 프로젝트 관리자 2차 결재 승인\n \n - 아래와 같이 `생성` 버튼을 클릭하게 되면 Namespace 신청 페이지로 이동합니다. 라는 안내 문구와 함께 확인 버튼을 통하여 신청 페이지로 이동할 수 있습니다.\n\n ![](img/namespace_create_button.png)\n\n2. 네임스페이스 신청은 아래 form을 작성 후 신청할 수 있으며 항목 선택 및 입력 후 확인 버튼을 클릭하면 네임스페이스 생성 신청이 완료 됩니다. 주요 선택 항목에 대한 설명은 아래와 같습니다.\n - Project: 네임스페이스가 속하게 될 프로젝트를 선택하는 항목입니다. \n - Cluster: 프로젝트 하위에 소속된 클러스터의 목록이며 실제 신청하는 네임스페이스가 배포되는 클러스터 입니다.\n - Namespace: 신청하는 네임스페이스의 이름입니다. 검색 버튼을 통하여 클러스터 하위에 동일한 이름의 네임스페이스가 존재하는지 여부를 체크합니다.\n - 사용목적: 네임스페이스가 사용되는 목적을 작성하는 항목입니다.\n - CPU: 네임스페이스에 할당할 CPU 용량입니다.\n - Memory(GiB): 네임스페이스에 할당할 메모리 용량입니다.\n - Storage(GiB): 네임스페이스에 할당할 스토리지 용량입니다.\n\n ![](img/namespace_application.png)\n\n3. 신청을 완료하면 신청관리 화면으로 이동하게 되며 내가 신청한 네임스페이스의 목록 및 결재 단계를 확인할 수 있습니다. 운영자 > 프로젝트 관리자의 1,2차 승인 후 네임스페이스가 추가되는것을 확인할 수 있습니다.\n\n ![](img/namespace_application_management.png)\n\n---",firstUser,namespaces,2. Namespace 생성 신청,https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/namespaces,"[https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/img/namespace_create_button.png, https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/img/namespace_application.png, https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/img/namespace_application_management.png]"
4,"1. 신청관리 화면에서 운영자, 프로젝트 관리자의 1,2차 승인이 완료되면 네임스페이스가 생성이 되며 해당 화면에서 확인할 수 있습니다.\n - 직접 생성한 프로젝트 및 하위 클러스터, 승인 완료된 네임스페이스를 상단에서 선택을 한 후 네임스페이스 메뉴에 진입하면 생성된 네임스페이스를 확인할 수 있습니다.\n - 직접 신청하여 생성된 네임스페이스내에서의 부여된 권한은 네임스페이스 관리자 권한을 갖게 됩니다.\n - 네임스페이스 생성 확인 및 특정 프로젝트 > 네임스페이스 하위에 유저의 권한이 할당되면 오픈된 메뉴에 한해서 각각의 기능을 수행할 수 있습니다. (<U>**각 수행 가능한 기능은 일반 사용자 매뉴얼 참조**</U>) \n\n ![](img/namespace_create_result.png)",firstUser,namespaces,3. Namespace 생성 확인,https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/namespaces,[https://doc.tg-cloud.co.kr/manual/console/firstUser/namespaces/img/namespace_create_result.png]
5,"# Project\n\n> 사용자에게 할당된 Project와 하위의 Namespace, 사용자 정보를 확인 할 수 있는 메뉴입니다. <br />\n> 새로운 Project를 생성 및 삭제 할 수 있고, Project 관리자나 사용자를 추가 및 삭제 할 수 있습니다.\n\n---",firstUser,project,__intro__,https://doc.tg-cloud.co.kr/manual/console/firstUser/project/project,[]
6,1. 좌측 메뉴 `Project` 메뉴 클릭\n\n\n ![](img/project_menu.png)\n\n\n - 좌측 메뉴 `Project` 메뉴 클릭 후 프로젝트 화면에 진입합니다.\n - 최초 로그인한 사용자에게 할당된 프로젝트가 없어 빈 리스트로 표현됩니다.\n\n---,firstUser,project,1. Project 메뉴 확인,https://doc.tg-cloud.co.kr/manual/console/firstUser/project/project,[https://doc.tg-cloud.co.kr/manual/console/firstUser/project/img/project_menu.png]
7,1. `생성` 버튼을 클릭하여 프로젝트를 생성할 수 있습니다.\n\n![](img/project_create_button.png)\n\n2. 프로젝트 이름만 입력 후 생성할 수 있습니다. 있으며 \n\n![](img/project_create.png)\n\n---,firstUser,project,2. Project 생성,https://doc.tg-cloud.co.kr/manual/console/firstUser/project/project,"[https://doc.tg-cloud.co.kr/manual/console/firstUser/project/img/project_create_button.png, https://doc.tg-cloud.co.kr/manual

## 1. 기본 유틸리티 함수들

In [12]:
import os
import re
from typing import List, Dict, Tuple, Optional
from pathlib import Path

In [42]:


# =============================================================================
# 1. 개선된 기본 유틸리티 함수들 (다계층 디렉토리 지원)
# =============================================================================

def extract_base_directory_name(root_dir: str) -> str:
    """
    루트 디렉토리 경로에서 마지막 디렉토리 이름만 추출합니다.
    
    Args:
        root_dir (str): 디렉토리 경로 (예: './manual/user/firstUser')
    
    Returns:
        str: 마지막 디렉토리 이름 (예: 'firstUser')
    """
    return os.path.basename(os.path.normpath(root_dir))

def find_markdown_files_recursive(root_dir: str) -> List[Tuple[str, str]]:
    """
    루트 디렉토리에서 재귀적으로 .md 파일을 찾습니다.
    
    Args:
        root_dir (str): 검색할 루트 디렉토리
    
    Returns:
        List[Tuple[str, str]]: (상대경로, 파일명) 튜플 리스트
    """
    markdown_files = []
    
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.md'):
                # 상대 경로 계산
                rel_path = os.path.relpath(root, root_dir)
                if rel_path == '.':
                    rel_path = ''
                
                # 파일명에서 확장자 제거
                file_base = os.path.splitext(file)[0]
                markdown_files.append((rel_path, file_base))

    return sorted(markdown_files)

def validate_markdown_structure(root_dir: str, rel_path: str, file_base: str) -> Tuple[bool, str, str]:
    """
    마크다운 파일 구조가 유효한지 검증합니다.
    
    Args:
        root_dir (str): 루트 디렉토리
        rel_path (str): 상대 경로
        file_base (str): 파일 기본명 (확장자 제외)
    
    Returns:
        Tuple[bool, str, str]: (유효성, md_path, img_dir_path)
    """
    if rel_path:
        full_path = os.path.join(root_dir, rel_path)
        md_path = os.path.join(full_path, f"{file_base}.md")
        img_dir = os.path.join(full_path, "img")
    else:
        md_path = os.path.join(root_dir, f"{file_base}.md")
        img_dir = os.path.join(root_dir, "img")
    
    is_valid = os.path.isfile(md_path)
    return is_valid, md_path, img_dir

def get_manual_type_from_path(rel_path: str, file_base: str) -> str:
    """
    경로와 파일명으로부터 매뉴얼 타입을 결정합니다.
    
    Args:
        rel_path (str): 상대 경로
        file_base (str): 파일 기본명
    
    Returns:
        str: 매뉴얼 타입
    """
    if rel_path:
        # 다계층인 경우: "accesscontrol/clusterrolebindings" -> "clusterrolebindings"
        return os.path.basename(rel_path)
    else:
        # 단일 계층인 경우: 파일명 사용
        return file_base

def is_image_file(filename: str) -> bool:
    """
    파일이 이미지 파일인지 확인합니다.
    
    Args:
        filename (str): 파일명
    
    Returns:
        bool: 이미지 파일 여부
    """
    image_extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.svg', '.webp')
    return filename.lower().endswith(image_extensions)

# 테스트
print("✅ 개선된 기본 유틸리티 함수들이 정의되었습니다.")

✅ 개선된 기본 유틸리티 함수들이 정의되었습니다.


## 2. 이미지 처리 관련 함수들

In [25]:
# =============================================================================
# 2. 개선된 이미지 처리 함수들 (다계층 지원)
# =============================================================================

def scan_image_directory(img_dir: str) -> List[str]:
    """
    이미지 디렉토리를 스캔하여 이미지 파일 목록을 반환합니다.
    
    Args:
        img_dir (str): 이미지 디렉토리 경로
    
    Returns:
        List[str]: 이미지 파일명 리스트 (정렬된 상태)
    """
    if not os.path.isdir(img_dir):
        return []
    
    image_files = []
    try:
        for filename in sorted(os.listdir(img_dir)):
            if is_image_file(filename):
                image_files.append(filename)
    except (OSError, PermissionError) as e:
        print(f"⚠️ 이미지 디렉토리 스캔 실패: {img_dir} - {e}")
        return []
    
    return image_files

def generate_image_urls_multilevel(image_files: List[str], base_dir_name: str, rel_path: str, manu_type: str) -> List[str]:
    """
    다계층 구조를 고려하여 이미지 URL을 생성합니다.
    
    Args:
        image_files (List[str]): 이미지 파일명 리스트
        base_dir_name (str): 기본 디렉토리 이름 (예: 'admin')
        rel_path (str): 상대 경로 (예: 'accesscontrol/clusterrolebindings')
        manu_type (str): 매뉴얼 타입 (예: 'clusterrolebindings')
    
    Returns:
        List[str]: 완전한 이미지 URL 리스트
    """
    base_url = "https://doc.tg-cloud.co.kr/manual/console"
    
    image_urls = []
    for img_file in image_files:
        if rel_path:
            # 다계층: /manual/console/admin/accesscontrol/clusterrolebindings/img/xxx.png
            url = f"{base_url}/{base_dir_name}/{rel_path}/img/{img_file}"
        else:
            # 단일계층: /manual/console/firstUser/approval/img/xxx.png
            url = f"{base_url}/{base_dir_name}/{manu_type}/img/{img_file}"
        
        image_urls.append(url)
    
    return image_urls

def process_manual_images_multilevel(img_dir: str, base_dir_name: str, rel_path: str, manu_type: str) -> List[str]:
    """
    다계층 매뉴얼의 이미지 처리를 종합적으로 수행합니다.
    
    Args:
        img_dir (str): 이미지 디렉토리 경로
        base_dir_name (str): 기본 디렉토리 이름
        rel_path (str): 상대 경로
        manu_type (str): 매뉴얼 타입
    
    Returns:
        List[str]: 완전한 이미지 URL 리스트
    """
    # 1. 이미지 파일 스캔
    image_files = scan_image_directory(img_dir)
    
    # 2. URL 생성 (다계층 지원)
    image_urls = generate_image_urls_multilevel(image_files, base_dir_name, rel_path, manu_type)
    
    return image_urls

# 테스트
print("✅ 개선된 이미지 처리 함수들이 정의되었습니다.")

✅ 개선된 이미지 처리 함수들이 정의되었습니다.


## 3. 메타데이터 생성 함수들

In [26]:
# =============================================================================
# 3. 개선된 메타데이터 생성 함수들 (다계층 지원)
# =============================================================================

def generate_source_url_multilevel(base_dir_name: str, rel_path: str, manu_type: str) -> str:
    """
    다계층 구조를 고려하여 매뉴얼의 소스 URL을 생성합니다.
    
    Args:
        base_dir_name (str): 기본 디렉토리 이름
        rel_path (str): 상대 경로
        manu_type (str): 매뉴얼 타입
    
    Returns:
        str: 완전한 소스 URL
    """
    base_url = "https://doc.tg-cloud.co.kr/manual/console"
    
    if rel_path:
        # 다계층: /manual/console/admin/accesscontrol/clusterrolebindings/clusterrolebindings
        return f"{base_url}/{base_dir_name}/{rel_path}/{manu_type}"
    else:
        # 단일계층: /manual/console/firstUser/approval/approval
        return f"{base_url}/{base_dir_name}/{manu_type}/{manu_type}"

def create_manual_metadata_multilevel(manual_type: str, manu_type: str, source_url: str, 
                                    image_urls: List[str], rel_path: str = "") -> Dict:
    """
    다계층 매뉴얼의 메타데이터 딕셔너리를 생성합니다.
    
    Args:
        manual_type (str): 상위 매뉴얼 타입 (예: 'admin')
        manu_type (str): 현재 매뉴얼 타입 (예: 'clusterrolebindings')
        source_url (str): 소스 URL
        image_urls (List[str]): 이미지 URL 리스트
        rel_path (str): 상대 경로 (카테고리 정보로 활용)
    
    Returns:
        Dict: 메타데이터 딕셔너리
    """
    metadata = {
        "manual_type": manual_type,
        "manu_type": manu_type,
        "source_url": source_url,
        "image_url": image_urls
    }
    
    # 다계층인 경우 카테고리 정보 추가
    if rel_path:
        # "accesscontrol/clusterrolebindings" -> "accesscontrol"
        category = rel_path.split('/')[0] if '/' in rel_path else rel_path
        metadata["category"] = category
        metadata["full_path"] = rel_path
    
    return metadata

def create_manual_entry_multilevel(manu_type: str, metadata: Dict, rel_path: str = "") -> Dict:
    """
    다계층 구조를 고려한 완전한 매뉴얼 엔트리를 생성합니다.
    
    Args:
        manu_type (str): 매뉴얼 타입
        metadata (Dict): 메타데이터 딕셔너리
        rel_path (str): 상대 경로
    
    Returns:
        Dict: 완전한 매뉴얼 엔트리
    """
    if rel_path:
        # 다계층인 경우 경로 정보 포함
        text = f"{rel_path}/{manu_type}.md"
    else:
        # 단일계층인 경우
        text = f"{manu_type}.md"
    
    return {
        "text": text,
        "metadata": metadata
    }

# 테스트
print("✅ 개선된 메타데이터 생성 함수들이 정의되었습니다.")

✅ 개선된 메타데이터 생성 함수들이 정의되었습니다.


## 4. 디렉토리 스캔 및 처리 함수들

In [27]:
# =============================================================================
# 4. 개선된 디렉토리 처리 함수들 (다계층 지원)
# =============================================================================

def analyze_directory_structure(root_dir: str) -> Dict[str, any]:
    """
    디렉토리 구조를 분석하여 통계 정보를 반환합니다.
    
    Args:
        root_dir (str): 분석할 루트 디렉토리
    
    Returns:
        Dict: 구조 분석 결과
    """
    markdown_files = find_markdown_files_recursive(root_dir)
    
    structure_info = {
        "total_markdown_files": len(markdown_files),
        "single_level": [],  # 단일 계층 파일들
        "multi_level": [],   # 다계층 파일들
        "categories": set(), # 카테고리 목록
        "depth_distribution": {}  # 깊이별 분포
    }
    
    for rel_path, file_base in markdown_files:
        if rel_path:
            # 다계층
            structure_info["multi_level"].append((rel_path, file_base))
            
            # 카테고리 추출
            category = rel_path.split('/')[0]
            structure_info["categories"].add(category)
            
            # 깊이 계산
            depth = len(rel_path.split('/'))
            structure_info["depth_distribution"][depth] = structure_info["depth_distribution"].get(depth, 0) + 1
        else:
            # 단일계층
            structure_info["single_level"].append(file_base)
            structure_info["depth_distribution"][0] = structure_info["depth_distribution"].get(0, 0) + 1
    
    structure_info["categories"] = sorted(list(structure_info["categories"]))
    
    return structure_info

def process_single_manual_multilevel(root_dir: str, rel_path: str, file_base: str, manual_type: str) -> Optional[Dict]:
    """
    다계층 구조를 지원하는 단일 매뉴얼 처리 함수입니다.
    
    Args:
        root_dir (str): 루트 디렉토리 경로
        rel_path (str): 상대 경로
        file_base (str): 파일 기본명
        manual_type (str): 상위 매뉴얼 타입
    
    Returns:
        Optional[Dict]: 매뉴얼 엔트리 또는 None (실패시)
    """
    # 1. 기본 정보 추출
    base_dir_name = extract_base_directory_name(root_dir)
    manu_type = get_manual_type_from_path(rel_path, file_base)
    
    # 2. 구조 검증
    is_valid, md_path, img_dir = validate_markdown_structure(root_dir, rel_path, file_base)
    
    if not is_valid:
        print(f"⚠️ 유효하지 않은 매뉴얼 구조: {md_path} (.md 파일 없음)")
        return None
    
    # 3. 이미지 처리 (다계층 지원)
    image_urls = process_manual_images_multilevel(img_dir, base_dir_name, rel_path, manu_type)
    
    # 4. 메타데이터 생성 (다계층 지원)
    source_url = generate_source_url_multilevel(base_dir_name, rel_path, manu_type)
    metadata = create_manual_metadata_multilevel(manual_type, manu_type, source_url, image_urls, rel_path)
    
    # 5. 최종 엔트리 생성 (다계층 지원)
    manual_entry = create_manual_entry_multilevel(manu_type, metadata, rel_path)
    
    return manual_entry

def get_processing_statistics(root_dir: str) -> Dict[str, any]:
    """
    처리 통계를 상세하게 반환합니다.
    
    Args:
        root_dir (str): 루트 디렉토리
    
    Returns:
        Dict: 상세 처리 통계
    """
    structure_info = analyze_directory_structure(root_dir)
    markdown_files = find_markdown_files_recursive(root_dir)
    
    stats = {
        "structure": structure_info,
        "processing": {
            "total_files": len(markdown_files),
            "valid_manuals": 0,
            "invalid_manuals": 0,
            "total_images": 0,
            "single_level_count": len(structure_info["single_level"]),
            "multi_level_count": len(structure_info["multi_level"])
        }
    }
    
    # 각 파일 검증
    for rel_path, file_base in markdown_files:
        is_valid, _, img_dir = validate_markdown_structure(root_dir, rel_path, file_base)
        
        if is_valid:
            stats["processing"]["valid_manuals"] += 1
            image_files = scan_image_directory(img_dir)
            stats["processing"]["total_images"] += len(image_files)
        else:
            stats["processing"]["invalid_manuals"] += 1
    
    return stats

# 테스트
print("✅ 개선된 디렉토리 처리 함수들이 정의되었습니다.")

✅ 개선된 디렉토리 처리 함수들이 정의되었습니다.


## 5. 메인 함수 (모듈화된 버전)

In [28]:
# =============================================================================
# 5. 개선된 메인 함수 (다계층 지원)
# =============================================================================

def generate_manual_index_multilevel(root_dir: str, manual_type: str = "user", verbose: bool = False) -> List[Dict]:
    """
    다계층 구조를 지원하는 매뉴얼 인덱스 생성 함수입니다.
    
    Args:
        root_dir (str): 메뉴얼 파일들이 있는 최상위 디렉토리 경로
        manual_type (str): 메뉴얼 종류 구분자 (기본값: "user")
        verbose (bool): 상세 로그 출력 여부
    
    Returns:
        List[Dict]: 메뉴얼 정보를 담은 딕셔너리들의 리스트
    """
    if verbose:
        print(f"🔍 다계층 매뉴얼 인덱스 생성 시작: {root_dir}")
        
        # 처리 전 상세 통계 출력
        stats = get_processing_statistics(root_dir)
        print(f"📊 디렉토리 구조 분석:")
        print(f"   • 총 마크다운 파일: {stats['structure']['total_markdown_files']}")
        print(f"   • 단일 계층 파일: {stats['processing']['single_level_count']}")
        print(f"   • 다계층 파일: {stats['processing']['multi_level_count']}")
        print(f"   • 발견된 카테고리: {', '.join(stats['structure']['categories'])}")
        print(f"   • 깊이별 분포: {stats['structure']['depth_distribution']}")
        print(f"   • 유효한 매뉴얼: {stats['processing']['valid_manuals']}")
        print(f"   • 무효한 매뉴얼: {stats['processing']['invalid_manuals']}")
        print(f"   • 총 이미지 파일: {stats['processing']['total_images']}")
        print()
    
    # 1. 마크다운 파일 재귀 검색
    markdown_files = find_markdown_files_recursive(root_dir)
    
    if not markdown_files:
        print(f"⚠️ 처리할 마크다운 파일이 없습니다: {root_dir}")
        return []
    
    # 2. 각 파일 처리
    result = []
    processed_count = 0
    skipped_count = 0
    
    for rel_path, file_base in markdown_files:
        if verbose:
            display_path = f"{rel_path}/{file_base}" if rel_path else file_base
            print(f"🔄 처리 중: {display_path}")
        
        manual_entry = process_single_manual_multilevel(root_dir, rel_path, file_base, manual_type)
        
        if manual_entry:
            result.append(manual_entry)
            processed_count += 1
            
            if verbose:
                image_count = len(manual_entry["metadata"]["image_url"])
                category_info = f" (카테고리: {manual_entry['metadata'].get('category', 'None')})" if 'category' in manual_entry['metadata'] else ""
                print(f"   ✅ 성공 - 이미지 {image_count}개{category_info}")
        else:
            skipped_count += 1
            if verbose:
                print(f"   ❌ 스킵됨")
    
    # 3. 최종 결과 출력
    if verbose:
        print(f"\n📋 다계층 매뉴얼 인덱스 생성 완료:")
        print(f"   • 처리된 매뉴얼: {processed_count}개")
        print(f"   • 스킵된 매뉴얼: {skipped_count}개")
        print(f"   • 총 생성된 엔트리: {len(result)}개")
        
        # 카테고리별 통계
        category_stats = {}
        for entry in result:
            category = entry["metadata"].get("category", "단일계층")
            category_stats[category] = category_stats.get(category, 0) + 1
        
        print(f"   • 카테고리별 분포:")
        for category, count in sorted(category_stats.items()):
            print(f"     - {category}: {count}개")
    
    return result

# 기존 함수와의 호환성을 위한 래퍼 함수 (다계층 지원)
def generate_manual_index(root_dir: str, manual_type: str = "user") -> List[Dict]:
    """
    기존 함수와 동일한 인터페이스를 제공하는 래퍼 함수입니다. (다계층 지원)
    """
    return generate_manual_index_multilevel(root_dir, manual_type, verbose=False)

# 테스트
print("✅ 개선된 메인 함수가 정의되었습니다.")
print("🎉 다계층 지원 모듈화된 함수들이 준비되었습니다!")

✅ 개선된 메인 함수가 정의되었습니다.
🎉 다계층 지원 모듈화된 함수들이 준비되었습니다!


## 6. 테스트 및 검증 코드


In [31]:
test_root = "./manual/user"

 # 1. 구조 분석
print(f"\n1️⃣ 디렉토리 구조 분석")
structure_info = analyze_directory_structure(test_root)
print(f"   • 총 마크다운 파일: {structure_info['total_markdown_files']}")
print(f"   • 단일계층: {len(structure_info['single_level'])}")
print(f"   • 다계층: {len(structure_info['multi_level'])}")
print(f"   • 카테고리: {', '.join(structure_info['categories']) if structure_info['categories'] else '없음'}")






1️⃣ 디렉토리 구조 분석
   • 총 마크다운 파일: 8
   • 단일계층: 0
   • 다계층: 8
   • 카테고리: accesscontrol, approval, bookmark, firstUser


In [38]:
  # 2. 샘플 파일 표시
print(f"\n2️⃣ 샘플 파일들")
markdown_files = find_markdown_files_recursive(test_root)
for i, (rel_path, file_base) in enumerate(markdown_files[:5]):  # 상위 5개만
    display_path = f"{rel_path}/{file_base}" if rel_path else file_base
    print(f"   • {display_path}")

if len(markdown_files) > 5:
    print(f"   • ... 외 {len(markdown_files) - 5}개")
        


2️⃣ 샘플 파일들
   • accesscontrol/clusterrolebindings/clusterrolebindings.md
   • accesscontrol/clusterroles/clusterroles.md
   • approval/approval.md
   • bookmark/bookmark.md
   • firstUser/approval/approval.md
   • ... 외 3개


In [43]:
# 3. 실제 처리 테스트
print(f"\n3️⃣ 매뉴얼 인덱스 생성 테스트")
manual_type = extract_base_directory_name(test_root)
result = generate_manual_index_multilevel(test_root, manual_type, verbose=True)
        
print(f"\n4️⃣ 결과 샘플")
if result:
    sample = result[0]
    print(f"   • 샘플 엔트리:")
    print(f"     - text: {sample['text']}")
    print(f"     - manu_type: {sample['metadata']['manu_type']}")
    print(f"     - category: {sample['metadata'].get('category', '없음')}")
    print(f"     - 이미지 수: {len(sample['metadata']['image_url'])}")
            
    if sample['metadata']['image_url']:
        print(f"     - 첫 번째 이미지: {sample['metadata']['image_url'][0]}")
    


3️⃣ 매뉴얼 인덱스 생성 테스트
🔍 다계층 매뉴얼 인덱스 생성 시작: ./manual/user
📊 디렉토리 구조 분석:
   • 총 마크다운 파일: 8
   • 단일 계층 파일: 0
   • 다계층 파일: 8
   • 발견된 카테고리: accesscontrol, approval, bookmark, firstUser
   • 깊이별 분포: {2: 6, 1: 2}
   • 유효한 매뉴얼: 8
   • 무효한 매뉴얼: 0
   • 총 이미지 파일: 59

🔄 처리 중: accesscontrol/clusterrolebindings/clusterrolebindings
   ✅ 성공 - 이미지 12개 (카테고리: accesscontrol)
🔄 처리 중: accesscontrol/clusterroles/clusterroles
   ✅ 성공 - 이미지 11개 (카테고리: accesscontrol)
🔄 처리 중: approval/approval
   ✅ 성공 - 이미지 8개 (카테고리: approval)
🔄 처리 중: bookmark/bookmark
   ✅ 성공 - 이미지 6개 (카테고리: bookmark)
🔄 처리 중: firstUser/approval/approval
   ✅ 성공 - 이미지 8개 (카테고리: firstUser)
🔄 처리 중: firstUser/login/login
   ✅ 성공 - 이미지 5개 (카테고리: firstUser)
🔄 처리 중: firstUser/namespaces/namespaces
   ✅ 성공 - 이미지 5개 (카테고리: firstUser)
🔄 처리 중: firstUser/project/project
   ✅ 성공 - 이미지 4개 (카테고리: firstUser)

📋 다계층 매뉴얼 인덱스 생성 완료:
   • 처리된 매뉴얼: 8개
   • 스킵된 매뉴얼: 0개
   • 총 생성된 엔트리: 8개
   • 카테고리별 분포:
     - accesscontrol: 2개
     - approval: 1개
     - bookmark: 1개

In [45]:
print(result[0])

{'text': 'accesscontrol/clusterrolebindings/clusterrolebindings.md', 'metadata': {'manual_type': 'user', 'manu_type': 'clusterrolebindings', 'source_url': 'https://doc.tg-cloud.co.kr/manual/console/user/accesscontrol/clusterrolebindings/clusterrolebindings', 'image_url': ['https://doc.tg-cloud.co.kr/manual/console/user/accesscontrol/clusterrolebindings/img/clusterrolebinding_binding_info.png', 'https://doc.tg-cloud.co.kr/manual/console/user/accesscontrol/clusterrolebindings/img/clusterrolebinding_create_result.png', 'https://doc.tg-cloud.co.kr/manual/console/user/accesscontrol/clusterrolebindings/img/clusterrolebinding_create_template.png', 'https://doc.tg-cloud.co.kr/manual/console/user/accesscontrol/clusterrolebindings/img/clusterrolebinding_create_yaml.png', 'https://doc.tg-cloud.co.kr/manual/console/user/accesscontrol/clusterrolebindings/img/clusterrolebinding_delete.png', 'https://doc.tg-cloud.co.kr/manual/console/user/accesscontrol/clusterrolebindings/img/clusterrolebinding_delet

### 1단계: 파일 시스템 모듈

In [56]:
import black
# 설치 확인
!pip list | grep black
!pip list | grep autopep8
# Black 포맷터 로드
%load_ext nb_black

# 또는 autopep8 사용하려면
%load_ext autopep8

ModuleNotFoundError: No module named 'black'

In [46]:
import os
from typing import List, Tuple, Optional
from pathlib import Path

In [47]:
def scan_directory(root_path: str) -> List[str]:
    """
    디렉토리를 스캔하여 .md 파일 경로를 찾습니다.
    
    Args:
        root_path (str): 스캔할 루트 디렉토리 경로
    
    Returns:
        List[str]: 발견된 .md 파일의 절대 경로 리스트
    
    Example:
        >>> scan_directory("./manual/user/firstUser")
        ['/path/to/manual/user/firstUser/login/login.md', 
         '/path/to/manual/user/firstUser/approval/approval.md']
    """
    md_files = []
    
    if not os.path.exists(root_path):
        print(f"⚠️ 경로가 존재하지 않습니다: {root_path}")
        return md_files
    
    for root, dirs, files in os.walk(root_path):
        for file in files:
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                md_files.append(os.path.abspath(file_path))
    
    return sorted(md_files)

In [60]:

test_path = "./manual/user"
md_files = scan_directory(test_path) 


print(f"   📁 스캔 경로: {test_path}")
print(f"   📄 발견된 .md 파일 수: {len(md_files)}")

if md_files:
    print("   📋 발견된 파일들:")
    for i, file_path in enumerate(md_files, 1):
        print(f"      {i}. {file_path}")
else:
    print("   ⚠️ .md 파일을 찾을 수 없습니다.")



   📁 스캔 경로: ./manual/user
   📄 발견된 .md 파일 수: 8
   📋 발견된 파일들:
      1. /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/accesscontrol/clusterrolebindings/clusterrolebindings.md
      2. /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/accesscontrol/clusterroles/clusterroles.md
      3. /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/approval/approval.md
      4. /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/bookmark/bookmark.md
      5. /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/approval/approval.md
      6. /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/login/login.md
      7. /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/namespaces/namespaces.md
      8. /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/project/project.md


In [58]:
def read_markdown_file(file_path: str) -> Optional[str]:
    """
    마크다운 파일의 내용을 읽습니다.
    
    Args:
        file_path (str): 읽을 마크다운 파일 경로
    
    Returns:
        Optional[str]: 파일 내용 (실패 시 None)
    
    Example:
        >>> content = read_markdown_file("./manual/user/firstUser/login/login.md")
        >>> print(content[:50])
        # Login
        
        ---
        ## 목차
        1. [로그인 페이지](#1-로그인-페이지)
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return None
    except UnicodeDecodeError:
        print(f"❌ 파일 인코딩 오류: {file_path}")
        return None
    except Exception as e:
        print(f"❌ 파일 읽기 오류: {file_path} - {str(e)}")
        return None

In [61]:
print("\n2️⃣ read_markdown_file 함수 테스트")
if md_files:
    test_file = md_files[0]  # 첫 번째 파일로 테스트
    print(f"   📖 테스트 파일: {test_file}")
    
    content = read_markdown_file(test_file)
    if content:
        print(f"   ✅ 파일 읽기 성공")
        print(f"   📊 파일 크기: {len(content)} 문자")
        print(f"   📝 첫 100자: {repr(content[:100])}")
    else:
        print("   ❌ 파일 읽기 실패")
else:
    print("   ⏭️ 테스트할 파일이 없습니다.")


2️⃣ read_markdown_file 함수 테스트
   📖 테스트 파일: /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/accesscontrol/clusterrolebindings/clusterrolebindings.md
   ✅ 파일 읽기 성공
   📊 파일 크기: 2378 문자
   📝 첫 100자: '# ClusterRoleBinding\n\n> 특정 클러스터 내에서 사용자, 그룹 또는 ServiceAccount에게 정의된 ClusterRole을 부여하는 역할을 합니다.\n\n---\n'


In [62]:
def extract_path_info(file_path: str, root_path: str) -> Tuple[str, str, str]:
    """
    파일 경로에서 경로 정보를 추출합니다.
    
    Args:
        file_path (str): 전체 파일 경로
        root_path (str): 루트 디렉토리 경로
    
    Returns:
        Tuple[str, str, str]: (상대경로, 파일명_확장자제거, 디렉토리명)
    
    Example:
        >>> extract_path_info("/path/manual/user/firstUser/login/login.md", "/path/manual/user/firstUser")
        ('login', 'login', 'login')
    """
    # 절대 경로로 변환
    abs_file_path = os.path.abspath(file_path)
    abs_root_path = os.path.abspath(root_path)
    
    # 상대 경로 계산
    rel_path = os.path.relpath(abs_file_path, abs_root_path)
    
    # 파일명과 확장자 분리
    file_name_with_ext = os.path.basename(abs_file_path)
    file_name = os.path.splitext(file_name_with_ext)[0]
    
    # 디렉토리명 추출
    dir_name = os.path.basename(os.path.dirname(abs_file_path))
    
    return (rel_path, file_name, dir_name)

In [63]:
# 테스트 3: extract_path_info 함수
print("\n3️⃣ extract_path_info 함수 테스트")
if md_files:
    test_file = md_files[0]
    rel_path, file_name, dir_name = extract_path_info(test_file, test_path)
    
    print(f"   📁 전체 경로: {test_file}")
    print(f"   📂 루트 경로: {test_path}")
    print(f"   📋 결과:")
    print(f"      • 상대경로: {rel_path}")
    print(f"      • 파일명: {file_name}")
    print(f"      • 디렉토리명: {dir_name}")
else:
    print("   ⏭️ 테스트할 파일이 없습니다.")

# 테스트 4: 에러 케이스 테스트
print("\n4️⃣ 에러 케이스 테스트")

# 존재하지 않는 디렉토리
print("   📁 존재하지 않는 디렉토리 테스트:")
empty_result = scan_directory("./nonexistent_directory")
print(f"      결과: {len(empty_result)}개 파일 발견")

# 존재하지 않는 파일
print("   📄 존재하지 않는 파일 테스트:")
none_result = read_markdown_file("./nonexistent_file.md")
print(f"      결과: {none_result}")

print("\n✅ 파일 시스템 모듈 테스트 완료!")


3️⃣ extract_path_info 함수 테스트
   📁 전체 경로: /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/accesscontrol/clusterrolebindings/clusterrolebindings.md
   📂 루트 경로: ./manual/user
   📋 결과:
      • 상대경로: accesscontrol/clusterrolebindings/clusterrolebindings.md
      • 파일명: clusterrolebindings
      • 디렉토리명: clusterrolebindings

4️⃣ 에러 케이스 테스트
   📁 존재하지 않는 디렉토리 테스트:
⚠️ 경로가 존재하지 않습니다: ./nonexistent_directory
      결과: 0개 파일 발견
   📄 존재하지 않는 파일 테스트:
❌ 파일을 찾을 수 없습니다: ./nonexistent_file.md
      결과: None

✅ 파일 시스템 모듈 테스트 완료!


### 2단계: 파싱 모듈

In [64]:
import re
from typing import List, Tuple, Optional

# =============================================================================
# 2단계: 파싱 모듈 (핵심 2개 함수만)
# =============================================================================

def parse_markdown_lines(content: str) -> List[str]:
    """
    마크다운 내용을 라인별로 분할합니다.
    
    Args:
        content (str): 마크다운 파일 내용
    
    Returns:
        List[str]: 라인별로 분할된 내용
    
    Example:
        >>> content = "# Title\n\n## Section\nContent here"
        >>> parse_markdown_lines(content)
        ['# Title', '', '## Section', 'Content here']
    """
    if not content:
        return []
    
    # 라인별로 분할하고 오른쪽 공백 제거
    lines = [line.rstrip() for line in content.split('\n')]
    return lines

In [65]:
def detect_header_level(line: str) -> Tuple[Optional[int], Optional[str]]:
    """
    라인에서 마크다운 헤더 레벨과 제목을 감지합니다.
    
    Args:
        line (str): 검사할 라인
    
    Returns:
        Tuple[Optional[int], Optional[str]]: (헤더레벨, 제목) 또는 (None, None)
    """
    # 마크다운 헤더 패턴 매칭 (# ~ ######)
    header_pattern = r'^(#{1,6})\s+(.+)$'
    match = re.match(header_pattern, line.strip())
    
    if match:
        header_level = len(match.group(1))  # # 개수
        header_title = match.group(2).strip()  # 제목 부분
        return (header_level, header_title)
    
    return (None, None)


In [66]:
def process_markdown_files(root_path: str) -> List[Dict[str, any]]:
    """
    1단계 함수들을 사용해서 마크다운 파일들을 파싱합니다.
    
    Args:
        root_path (str): 스캔할 루트 디렉토리 경로
    
    Returns:
        List[Dict]: 파싱된 파일 정보 리스트
        각 딕셔너리는 다음 키를 포함:
        - 'file_path': 파일 경로
        - 'rel_path': 상대 경로  
        - 'file_name': 파일명
        - 'dir_name': 디렉토리명
        - 'lines': 파싱된 라인 리스트
        - 'headers': 감지된 헤더 정보 리스트
    """
    # 1단계 함수 사용: 파일 스캔
    md_files = scan_directory(root_path)
    
    parsed_files = []
    
    for file_path in md_files:
        print(f"📖 파싱 중: {file_path}")
        
        # 1단계 함수 사용: 파일 읽기
        content = read_markdown_file(file_path)
        if content is None:
            continue
            
        # 1단계 함수 사용: 경로 정보 추출
        rel_path, file_name, dir_name = extract_path_info(file_path, root_path)
        
        # 2단계 함수 사용: 라인별 파싱
        lines = parse_markdown_lines(content)
        
        # 2단계 함수 사용: 헤더 감지
        headers = []
        for line_num, line in enumerate(lines):
            level, title = detect_header_level(line)
            if level is not None:
                headers.append({
                    'line_num': line_num,
                    'level': level,
                    'title': title,
                    'line': line
                })
        
        # 파싱 결과 저장
        parsed_file = {
            'file_path': file_path,
            'rel_path': rel_path,
            'file_name': file_name,
            'dir_name': dir_name,
            'lines': lines,
            'headers': headers,
            'total_lines': len(lines),
            'header_count': len(headers)
        }
        
        parsed_files.append(parsed_file)
    
    return parsed_files

In [70]:
test_path = "./manual/user"

print(f"\n📁 테스트 경로: {test_path}")

# 통합 처리 함수 실행
parsed_files = process_markdown_files(test_path)

print(f"\n📊 파싱 결과:")
print(f"   • 처리된 파일 수: {len(parsed_files)}")

# 각 파일별 상세 정보
for i, parsed_file in enumerate(parsed_files, 1):
    print(f"\n📋 파일 {i}: {parsed_file['file_name']}.md")
    print(f"   • 파일 경로: {parsed_file['file_path']}")
    print(f"   • 상대 경로: {parsed_file['rel_path']}")
    print(f"   • 디렉토리: {parsed_file['dir_name']}")
    print(f"   • 총 라인 수: {parsed_file['total_lines']}")
    print(f"   • 헤더 수: {parsed_file['header_count']}")
    
    # 헤더 정보 출력
    if parsed_file['headers']:
        print(f"   • 헤더 목록:")
        for header in parsed_file['headers']:
            indent = "  " * (header['level'] - 1)
            print(f"     {indent}└ 레벨 {header['level']}: {header['title']} (라인 {header['line_num'] + 1})")
    else:
        print(f"   • 헤더 없음")
    
    # 첫 5줄 미리보기
    print(f"   • 내용 미리보기 (첫 5줄):")
    for j, line in enumerate(parsed_file['lines'][:5]):
        print(f"     {j+1}: {repr(line)}")

print("\n✅ 통합 파싱 모듈 테스트 완료!")


📁 테스트 경로: ./manual/user
📖 파싱 중: /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/accesscontrol/clusterrolebindings/clusterrolebindings.md
📖 파싱 중: /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/accesscontrol/clusterroles/clusterroles.md
📖 파싱 중: /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/approval/approval.md
📖 파싱 중: /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/bookmark/bookmark.md
📖 파싱 중: /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/approval/approval.md
📖 파싱 중: /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/login/login.md
📖 파싱 중: /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/namespaces/namespaces.md
📖 파싱 중: /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/project/project.md

📊 파싱 결과:
   • 처리된 파일 수: 8

📋 파일 1: clusterrolebindings.md
   • 파일 경로: /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/accesscontrol/clusterrolebindings/clusterrolebindings.md
   • 상대 경로: accesscontrol/

### 3단계: 섹션 처리 모듈

In [68]:
from typing import List, Dict, Optional, Any

# =============================================================================
# 3단계: 섹션 처리 모듈 (2단계 출력을 입력으로 받음)
# =============================================================================

def should_skip_section(section_title: str) -> bool:
    """
    섹션을 스킵해야 하는지 판단합니다.
    
    Args:
        section_title (str): detect_header_level()에서 받은 섹션 제목
    
    Returns:
        bool: 스킵해야 하면 True, 처리해야 하면 False
    
    Example:
        >>> should_skip_section("목차")
        True
        >>> should_skip_section("1. 로그인 페이지")
        False
    """
    skip_patterns = [
        "목차",
        "**목차**",
        "table of contents",
        "toc",
        "contents"
    ]
    
    # 대소문자 구분 없이 비교
    section_lower = section_title.lower().strip()
    
    for pattern in skip_patterns:
        if pattern.lower() in section_lower:
            return True
    
    return False

def create_section(header: Dict[str, Any], content: List[str]) -> Dict[str, Any]:
    """
    헤더 정보와 내용을 받아서 섹션 객체를 생성합니다.
    
    Args:
        header (Dict): detect_header_level()에서 받은 헤더 정보
                      {'line_num': int, 'level': int, 'title': str}
        content (List[str]): parse_markdown_lines()에서 받은 해당 섹션의 내용 라인들
    
    Returns:
        Dict[str, Any]: 섹션 정보
        {
            'level': int,           # 헤더 레벨
            'title': str,           # 섹션 제목
            'content': List[str],   # 섹션 내용 (빈 라인 제거됨)
            'line_start': int,      # 시작 라인 번호
            'line_end': int,        # 종료 라인 번호
            'content_length': int,  # 내용 길이
            'is_empty': bool        # 빈 섹션 여부
        }
    
    Example:
        >>> header = {'line_num': 5, 'level': 2, 'title': '1. 로그인 페이지'}
        >>> content = ['브라우저에서 접속합니다.', '', '로그인 화면이 나타납니다.']
        >>> create_section(header, content)
        {'level': 2, 'title': '1. 로그인 페이지', 'content': [...], ...}
    """
    # 빈 라인 제거 및 내용 정리
    cleaned_content = []
    for line in content:
        line_stripped = line.strip()
        if line_stripped:  # 빈 라인이 아닌 경우만 추가
            cleaned_content.append(line)
    
    # 섹션 정보 생성
    section = {
        'level': header['level'],
        'title': header['title'],
        'content': cleaned_content,
        'line_start': header['line_num'],
        'line_end': header['line_num'] + len(content) - 1,
        'content_length': len(cleaned_content),
        'is_empty': len(cleaned_content) == 0
    }
    
    return section

def extract_sections_from_parsed_data(lines: List[str], headers: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    2단계에서 받은 파싱 데이터로부터 섹션들을 추출합니다.
    
    Args:
        lines (List[str]): parse_markdown_lines()에서 받은 라인 리스트
        headers (List[Dict]): detect_header_level()에서 받은 헤더 리스트
    
    Returns:
        List[Dict[str, Any]]: 처리된 섹션 리스트 (스킵된 섹션 제외, 레벨 2만 포함)
    """
    sections = []
    
    # 레벨 2 헤더만 필터링
    level2_headers = [h for h in headers if h['level'] == 2]
    
    for i, header in enumerate(level2_headers):
        # 스킵 대상 섹션 확인
        if should_skip_section(header['title']):
            print(f"   ⏭️ 스킵: {header['title']}")
            continue
        
        # 현재 섹션의 시작과 끝 라인 계산
        start_line = header['line_num'] + 1  # 헤더 다음 라인부터
        
        if i + 1 < len(level2_headers):
            # 다음 헤더가 있는 경우: 다음 헤더 직전까지
            end_line = level2_headers[i + 1]['line_num']
        else:
            # 마지막 헤더인 경우: 파일 끝까지
            end_line = len(lines)
        
        # 섹션 내용 추출
        section_content = lines[start_line:end_line]
        
        # 섹션 생성
        section = create_section(header, section_content)
        
        # 빈 섹션이 아닌 경우만 추가
        if not section['is_empty']:
            sections.append(section)
            print(f"   ✅ 생성: {section['title']} ({section['content_length']} 라인)")
        else:
            print(f"   ⚠️ 빈 섹션 제외: {section['title']}")
    
    return sections

In [ ]:
test_path = "./manual/user"
